In [1]:
import pandas as pd
from google.colab import drive

In [2]:
drive.mount('/content/gdrive')
DATADIR = '/content/gdrive/MyDrive/KaggleStoreSales/'

Mounted at /content/gdrive


In [3]:
dfTransactions = pd.read_csv(DATADIR + 'transactions.csv')
dfOil = pd.read_csv(DATADIR + 'oil.csv')
dfTest = pd.read_csv(DATADIR + 'test.csv')
dfStores = pd.read_csv(DATADIR + 'stores.csv')
dfHolidayEvents = pd.read_csv(DATADIR + 'holidays_events.csv')
dfHolidayEvents.drop('transferred', inplace=True, axis=1)
dfHolidayEvents.drop('description', inplace=True, axis=1)
newColumns = dfHolidayEvents.columns.values
newColumns[1] = 'holiday_type'
dfHolidayEvents.columns = newColumns

In [4]:
#combining all the data
dfTestMerge = dfTest.merge(dfStores, left_on=['store_nbr'], right_on=['store_nbr'], how='inner')
dfTestMerge = dfTestMerge.merge(dfOil, left_on=['date'], right_on=['date'], how='left')
dfTestMerge = dfTestMerge.fillna(axis=0, method='bfill')
dfTestMerge = dfTestMerge.merge(dfTransactions, left_on=['date','store_nbr'], right_on=['date','store_nbr'], how='left')
dfTestMerge = dfTestMerge.fillna(0)
print(dfTestMerge.isna().mean().round(4) * 100)
print(dfTestMerge.info())

id              0.0
date            0.0
store_nbr       0.0
family          0.0
onpromotion     0.0
city            0.0
state           0.0
type            0.0
cluster         0.0
dcoilwtico      0.0
transactions    0.0
dtype: float64
<class 'pandas.core.frame.DataFrame'>
Int64Index: 28512 entries, 0 to 28511
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   id            28512 non-null  int64  
 1   date          28512 non-null  object 
 2   store_nbr     28512 non-null  int64  
 3   family        28512 non-null  object 
 4   onpromotion   28512 non-null  int64  
 5   city          28512 non-null  object 
 6   state         28512 non-null  object 
 7   type          28512 non-null  object 
 8   cluster       28512 non-null  int64  
 9   dcoilwtico    28512 non-null  float64
 10  transactions  28512 non-null  float64
dtypes: float64(2), int64(4), object(5)
memory usage: 2.6+ MB
None


In [5]:
dfHolidayLocal = dfHolidayEvents[dfHolidayEvents['locale'] == 'Local']
print(dfHolidayLocal)
dfTrainMergeHolidayLocal = dfTestMerge.merge(dfHolidayLocal, left_on=['date','city'], right_on=['date','locale_name'], how='left')
dfTrainMergeHolidayLocal = dfTrainMergeHolidayLocal.drop_duplicates(subset=['id'], keep='first', inplace=False)
print(dfTrainMergeHolidayLocal.info())

           date holiday_type locale locale_name
0    2012-03-02      Holiday  Local       Manta
2    2012-04-12      Holiday  Local      Cuenca
3    2012-04-14      Holiday  Local    Libertad
4    2012-04-21      Holiday  Local    Riobamba
5    2012-05-12      Holiday  Local        Puyo
..          ...          ...    ...         ...
339  2017-12-05   Additional  Local       Quito
340  2017-12-06      Holiday  Local       Quito
341  2017-12-08      Holiday  Local        Loja
342  2017-12-08     Transfer  Local       Quito
344  2017-12-22      Holiday  Local     Salinas

[152 rows x 4 columns]
<class 'pandas.core.frame.DataFrame'>
Int64Index: 28512 entries, 0 to 28511
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   id            28512 non-null  int64  
 1   date          28512 non-null  object 
 2   store_nbr     28512 non-null  int64  
 3   family        28512 non-null  object 
 4   onpromotion   28512 non-null

In [6]:
dfHolidayRegional = dfHolidayEvents[dfHolidayEvents['locale'] == 'Regional']
dfTrainMergeHolidayRegional = dfTrainMergeHolidayLocal.merge(dfHolidayRegional, left_on=['date','state'], right_on=['date','locale_name'], how='left')
dfTrainMergeHolidayRegional = dfTrainMergeHolidayRegional.drop_duplicates(subset=['id'], keep='first', inplace=False)
print(dfTrainMergeHolidayRegional.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 28512 entries, 0 to 28511
Data columns (total 17 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   id              28512 non-null  int64  
 1   date            28512 non-null  object 
 2   store_nbr       28512 non-null  int64  
 3   family          28512 non-null  object 
 4   onpromotion     28512 non-null  int64  
 5   city            28512 non-null  object 
 6   state           28512 non-null  object 
 7   type            28512 non-null  object 
 8   cluster         28512 non-null  int64  
 9   dcoilwtico      28512 non-null  float64
 10  transactions    28512 non-null  float64
 11  holiday_type_x  66 non-null     object 
 12  locale_x        66 non-null     object 
 13  locale_name_x   66 non-null     object 
 14  holiday_type_y  0 non-null      object 
 15  locale_y        0 non-null      object 
 16  locale_name_y   0 non-null      object 
dtypes: float64(2), int64(4), object

In [7]:
dfHolidayNational = dfHolidayEvents[dfHolidayEvents['locale'] == 'National']
dfTrainMergeHolidayNational = dfTrainMergeHolidayRegional.merge(dfHolidayNational, left_on=['date'], right_on=['date'], how='left')
dfTrainMergeHolidayNational = dfTrainMergeHolidayNational.drop_duplicates(subset=['id'], keep='first', inplace=False)
print(dfTrainMergeHolidayNational.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 28512 entries, 0 to 28511
Data columns (total 20 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   id              28512 non-null  int64  
 1   date            28512 non-null  object 
 2   store_nbr       28512 non-null  int64  
 3   family          28512 non-null  object 
 4   onpromotion     28512 non-null  int64  
 5   city            28512 non-null  object 
 6   state           28512 non-null  object 
 7   type            28512 non-null  object 
 8   cluster         28512 non-null  int64  
 9   dcoilwtico      28512 non-null  float64
 10  transactions    28512 non-null  float64
 11  holiday_type_x  66 non-null     object 
 12  locale_x        66 non-null     object 
 13  locale_name_x   66 non-null     object 
 14  holiday_type_y  0 non-null      object 
 15  locale_y        0 non-null      object 
 16  locale_name_y   0 non-null      object 
 17  holiday_type    0 non-null     

In [8]:
dfTrainMergeHolidayNational['holiday_type'] = dfTrainMergeHolidayNational['holiday_type'].astype(str) + dfTrainMergeHolidayNational['holiday_type_x'].astype(str) + dfTrainMergeHolidayNational['holiday_type_y'].astype(str)
dfTrainMergeHolidayNational.drop('holiday_type_x', inplace=True, axis=1)
dfTrainMergeHolidayNational.drop('holiday_type_y', inplace=True, axis=1)
dfTrainMergeHolidayNational.drop('locale', inplace=True, axis=1)
dfTrainMergeHolidayNational.drop('locale_x', inplace=True, axis=1)
dfTrainMergeHolidayNational.drop('locale_y', inplace=True, axis=1)
dfTrainMergeHolidayNational.drop('locale_name', inplace=True, axis=1)
dfTrainMergeHolidayNational.drop('locale_name_x', inplace=True, axis=1)
dfTrainMergeHolidayNational.drop('locale_name_y', inplace=True, axis=1)
dfTrainMergeHolidayNational = dfTrainMergeHolidayNational.fillna('NONE')
print(dfTrainMergeHolidayNational.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 28512 entries, 0 to 28511
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   id            28512 non-null  int64  
 1   date          28512 non-null  object 
 2   store_nbr     28512 non-null  int64  
 3   family        28512 non-null  object 
 4   onpromotion   28512 non-null  int64  
 5   city          28512 non-null  object 
 6   state         28512 non-null  object 
 7   type          28512 non-null  object 
 8   cluster       28512 non-null  int64  
 9   dcoilwtico    28512 non-null  float64
 10  transactions  28512 non-null  float64
 11  holiday_type  28512 non-null  object 
dtypes: float64(2), int64(4), object(6)
memory usage: 2.8+ MB
None


In [11]:
#Label Encoding
from sklearn.preprocessing import OrdinalEncoder
encoder = OrdinalEncoder()
test = dfTrainMergeHolidayNational
encoder.fit(test[["family", "city", 'state', 'type', 'holiday_type']])
test[["family", "city", 'state', 'type', 'holiday_type']] = encoder.transform(test[["family", "city", 'state', 'type', 'holiday_type']])

In [ ]:
#Dummy Variables
test = dfTrainMergeHolidayNational
test = pd.get_dummies(test, columns=["family", "city", 'state', 'type', 'holiday_type'], prefix=["family", "city",'state', 'type', 'holiday_type'])
print(test.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 28512 entries, 0 to 28511
Data columns (total 85 columns):
 #   Column                                Non-Null Count  Dtype  
---  ------                                --------------  -----  
 0   id                                    28512 non-null  int64  
 1   date                                  28512 non-null  object 
 2   store_nbr                             28512 non-null  int64  
 3   onpromotion                           28512 non-null  int64  
 4   cluster                               28512 non-null  int64  
 5   dcoilwtico                            28512 non-null  float64
 6   transactions                          28512 non-null  float64
 7   family_AUTOMOTIVE                     28512 non-null  uint8  
 8   family_BABY CARE                      28512 non-null  uint8  
 9   family_BEAUTY                         28512 non-null  uint8  
 10  family_BEVERAGES                      28512 non-null  uint8  
 11  family_BOOKS   

In [12]:
import datetime as dt
test['date'] = test['date'].astype('datetime64[ns]')
test['date'] = pd.to_datetime(test['date'])
test['date']=test['date'].map(dt.datetime.toordinal)

In [13]:
test.to_csv(DATADIR + 'testSupplementaryLabelEncoding.csv', index=False)